# Which number is bigger?

In this example we will create an artificial problem to demonstrate different outputs from Z3. In this problem we have two variables, called A and B. Each variable is equal to either 1 or 2, and we want to know whether there are specific values each can have. The aim is to demonstrate how to do basic actions in Z3; creating variables, defining constraints, and asking a solver to find an assignment that is compatible. 

## Getting started

In this section we will be using Z3 as a library. To start we will need to download the files if they are not already available. These first cells download Z3 as a zip, extract it, and load it into memory. We then reference the extracted file and open it as a module.

** If the first cell does not run, manually download z3 from the link for your operating system and unzip it to a folder with the notebooks called "z3" **

In [ ]:
#r "System.IO.Compression.FileSystem.dll"

open System
open System.IO
open System.IO.Compression
open System.Net

//Specify Tls version to avoid cryptic connection errors
System.Net.ServicePointManager.SecurityProtocol <- SecurityProtocolType.Tls12 ||| SecurityProtocolType.Tls11 

let wc = new WebClient()

type OS =
        | OSX            
        | Windows
        | Linux

let getOS = 
        match int Environment.OSVersion.Platform with
        | 4 | 128 -> Linux
        | 6       -> OSX
        | _       -> Windows

if  true <> System.IO.File.Exists("z3/LICENSE.txt") then 
    match getOS with
    | Linux ->  wc.DownloadFile("https://github.com/Z3Prover/z3/releases/download/z3-4.6.0/z3-4.6.0-x64-ubuntu-16.04.zip", @"z3.zip")
                //This will take a while
                ZipFile.ExtractToDirectory("z3.zip", ".") 
                System.IO.Directory.Move("z3-4.6.0-x64-ubuntu-16.04","z3")
    | Windows ->wc.DownloadFile("https://github.com/Z3Prover/z3/releases/download/z3-4.6.0/z3-4.6.0-x64-win.zip", @"z3.zip")
                //This will take a while
                ZipFile.ExtractToDirectory("z3.zip", ".") 
                System.IO.Directory.Move("z3-4.6.0-x64-win","z3")

    | _ -> ()

In [ ]:
#r "z3/bin/Microsoft.Z3.dll"

In [ ]:
open Microsoft.Z3

## Initiating Z3 and Defining variables 

The main function contains the basic constraints that define what A and B must be. However, to demonstrate the outputs from Z3 we have created three functions that include different final constraints.

Firstly, we establish a context with the line

In [ ]:
let ctx = new Context()

You can think of a context as a palette of different constraints and types you can use to describe your model. It provides the expressions you need to describe your model in an appropriate type for Z3. Typically programmers will wrap common functions for a context in dedicated functions to maintain commmon F# stylings, but in most of the tutorials here we use the context directly. In the Suduko example below we show how we can wrap the functions.

We can then define elements using the context. We can define integer constant with the names "A" and "B" as

In [ ]:
let A = ctx.MkIntConst("A")
let B = ctx.MkIntConst("B")

This tells Z3 that A is a constant integer; we also want to specify that it is either equal to one or two. We need to specify these numbers through the context as below

In [ ]:
let zOne = ctx.MkInt(1)
let zTwo = ctx.MkInt(2)

## Defining and enforcing constraints
We can now use these variables in F# to make more complex expressions involving them. We know that either A or B can be one or two, so we need to specify that in through Z3. 

In [ ]:
let constraints = [|
    ctx.MkOr([|ctx.MkEq(A,zOne);ctx.MkEq(A,zTwo)|]);
    ctx.MkOr([|ctx.MkEq(B,zOne);ctx.MkEq(B,zTwo)|]);
    |]

This array contains a line that states that A=1 using MkEq, or A=2 using the variables we have previously defined. We put this term, and another term referencing B into an array called constraints.

These constraints define the problem, but we need to tell Z3 to use them. To do this we need a Solver, to which we add the constraints before asking for a solution. We create a solver, then add that all constraints must in the array must be respected by using an "And" term

In [ ]:
let s = ctx.MkSolver()
s.Add(ctx.MkAnd(constraints))

## Testing models

Now we can start testing! Three functions run different tests-

* *sanity_check* doesn't add anything but shows that the model is sound
* *paradox* adds constraints that cannot be satisfied and should show that
* *answer* addresses a "real" question- if A is less than B, is there a solution?


In [ ]:
let sanity_check (ctx:Context) (s:Solver) (v:IntExpr []) =
    s.Push()
    let r = s.Check()
    match r with 
    | Status.UNSATISFIABLE ->
        s.Pop()
        sprintf "FAIL sanity test- unsat\n"
    | Status.SATISFIABLE -> 
        let a = sprintf "PASS sanity test\nA %O B %O\n" (s.Model.ConstInterp(v.[0])) (s.Model.ConstInterp(v.[1]))
        s.Pop()
        a
    | _ -> failwith "Unknown fail"

let paradox (ctx:Context) (s:Solver) (variables:IntExpr []) =
    s.Push()
    let additionalConstraints = [|ctx.MkAnd([|ctx.MkEq(variables.[0],ctx.MkInt(2));ctx.MkEq(variables.[0],ctx.MkInt(1))|])|]
    s.Add(additionalConstraints)
    let r = s.Check()
    match r with 
    | Status.UNSATISFIABLE ->
        s.Pop()
        sprintf "PASS paradox test- unsat\n"
    | Status.SATISFIABLE -> 
         let result = sprintf "FAIL sanity test\nA %O B %O\n" (s.Model.ConstInterp(variables.[0])) (s.Model.ConstInterp(variables.[1]))
         s.Pop()
         result
    | _ -> failwith "Unknown fail"

let answer (ctx:Context) (s:Solver) (variables:IntExpr []) =
    s.Push()
    let additionalConstraints = [|ctx.MkLt(variables.[0],variables.[1])|]
    s.Add(additionalConstraints)
    let r = s.Check()
    match r with 
    | Status.UNSATISFIABLE ->
        s.Pop()
        "No answer- unsat\n"
    | Status.SATISFIABLE -> 
        let aState = s.Model.ConstInterp(variables.[0])
        let bState = s.Model.ConstInterp(variables.[1])
        let result = sprintf  "Answer- sat\nA %O B %O\n" aState bState
        s.Add (ctx.MkNot(ctx.MkAnd(ctx.MkEq(variables.[0],aState),ctx.MkEq(variables.[1],bState))))
        let r' = s.Check()
        match r' with
        | Status.UNSATISFIABLE ->
            s.Pop()
            result + "No other solutions\n"
        | Status.SATISFIABLE ->
            s.Pop()
            result + (sprintf "Alternative answer\nA %O B %O\n" (s.Model.ConstInterp(variables.[0])) (s.Model.ConstInterp(variables.[1])))
        | _ -> failwith "Unknown fail"
    | _ -> failwith "Unknown fail"

In each case we copy the solver, test the model, and then throw away the results. This allows us to add different, contradictory constraints without altering the other questions. To do this, we run three commands

* s.Push()
> Copies the system. Any new constraints you add will only be added to the copy, and any tests you perform will only be done on the working copy.

* s.Check()
> Asks Z3 to return an answer. The result can be tested using pattern matching as satisfiable (there is a solution), unsatisfiable (there is no solution) or unknown (something went wrong).

* s.Pop()
> Discards the working copy and reverts to the original copy.

Looking in sanity_check we copy the model, and test without adding any additional constraints. We then test the result and find that it is satisfiable. We then draw out the solution using

s.Model.ConstInterp
We can then print this to the terminal before discarding the result.

The function paradox is similar, but we add an additional, impossible to satisfy constraint

ctx.MkAnd([|ctx.MkEq(variables.[0],ctx.MkInt(2));ctx.MkEq(variables.[0],ctx.MkInt(1))|])
This says that A is equal to 2 and A is equal to 1. We then test, and should find that the result is unsatisfiable.

Finally the function answer adds a constraint that specifies that A is less than B. We can then retest and find the solution. MkLt is the expression used, meaning make less than (i.e. <), and there are other functions like MkLe (make less than or equal), MkGt, and MkGe.

ctx.MkLt(variables.[0],variables.[1])
Note that because we have used Push() and Pop() in each function, the constraints in paradox and answer functions do not clash; in each case we have only added them to a working copy that is discarded at the end of the work.

We do a further test in answer; we look at the solution that Check() gives us, and ask for an alternative answer. This is done by adding a further constraint to the solver, excluding the observed result from the solver, and running Check() again.

Finally, we run each of these checks and examine the outcome

In [ ]:
//Must be satisfiable
printf "%s" (sanity_check ctx s [|A;B|])
//Paradox- must not be satisfiable
printf "%s" (paradox ctx s [|A;B|])
//A is less than B- What is the answer?
printf "%s" (answer ctx s [|A;B|])